# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/song_data')

In [5]:
type(song_files)

list

In [6]:
song_files[0]

'/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [7]:
song_files[1]

'/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/song_data/A/A/A/TRAAARJ128F9320760.json'

In [8]:
import json
for i in song_files:
    with open(i, 'r') as file:
        json_data = json.load(file)
        #data.append(json_data)

In [9]:
type(json_data)

dict

In [10]:
json_data

{'num_songs': 1,
 'artist_id': 'ARGCY1Y1187B9A4FA5',
 'artist_latitude': 36.16778,
 'artist_longitude': -86.77836,
 'artist_location': 'Nashville, TN.',
 'artist_name': 'Gloriana',
 'song_id': 'SOQOTLQ12AB01868D0',
 'title': 'Clementina Santafè',
 'duration': 153.33832,
 'year': 0}

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_files = get_files('/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/song_data')
song_files[0]

'/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [12]:
import json
data = []
for i in song_files:
    with open(i, 'r') as file:
        json_data = json.load(file)
    data.append(json_data)
        
df = pd.DataFrame.from_records(data)
df.head(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [13]:
df.drop(['num_songs','artist_latitude',\
        'artist_longitude','artist_location','artist_name'],
        axis=1,inplace=True)
df.head()

,artist_id,song_id,title,duration,year
0,AR7G5I41187FB4CE6C,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,AR8ZCNI1187B9A069B,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,ARXR32B1187FB57099,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,AR10USD1187B99F3F1,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,ARGSJW91187B9B1D6B,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [14]:
df = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
df.head(3)

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
1,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
2,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


In [15]:
df_values = df.values
df_values[0]

array(['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C',
       1982, 233.40363], dtype=object)

In [16]:
song_data = list(df_values[0])

In [17]:
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [18]:
cur.execute(song_table_insert, song_data)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
import json
data = []
for i in song_files:
    with open(i, 'r') as file:
        json_data = json.load(file)
    data.append(json_data)
        
df2 = pd.DataFrame.from_records(data)
df2.head(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [20]:
df2 = df2[['artist_id', 'artist_name', 'artist_location',\
           'artist_latitude', 'artist_longitude']]
df2.head(2)

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN


In [21]:
df2_values = df2.values
artist_data = list(df2_values[0])
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [22]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [23]:
log_files = get_files('/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/log_data/')
log_files[0]

'/Users/tatianatikhonova/tati/Udacity Data Engineering - Project1/data/log_data/2018/11/2018-11-11-events.json'

In [24]:
more_data = []
first_file = log_files[0]
#for i in log_files:
for line in open(first_file,'r'):
     more_data.append(json.loads(line))
        
df3 = pd.DataFrame.from_records(more_data)
df3.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [25]:
more_data[0]

{'artist': 'Frumpies',
 'auth': 'Logged In',
 'firstName': 'Anabelle',
 'gender': 'F',
 'itemInSession': 0,
 'lastName': 'Simpson',
 'length': 134.47791,
 'level': 'free',
 'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'method': 'PUT',
 'page': 'NextSong',
 'registration': 1541044398796.0,
 'sessionId': 455,
 'song': 'Fuck Kitty',
 'status': 200,
 'ts': 1541903636796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"',
 'userId': '69'}

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [26]:
df3 = df3.loc[df3['page'] == 'NextSong']
df3.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [27]:
df3['ts'] = pd.to_datetime(df3['ts'])
df3.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1970-01-01 00:25:41.903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1970-01-01 00:25:41.903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [28]:
df3['ts'].tolist()[0:3]

[Timestamp('1970-01-01 00:25:41.903636796'),
 Timestamp('1970-01-01 00:25:41.903770796'),
 Timestamp('1970-01-01 00:25:41.904034796')]

In [29]:
#timestamp, hour, day, week of year, month, year, and weekday
time_df = df3[['ts']]
time_df['timestamp'] = pd.to_datetime(time_df['ts']).dt.time 
time_df['hour'] = pd.to_datetime(time_df['ts']).dt.hour
time_df['day'] = pd.to_datetime(time_df['ts']).dt.day
time_df['week'] = pd.to_datetime(time_df['ts']).dt.week
time_df['month'] = pd.to_datetime(time_df['ts']).dt.month
time_df['year'] = pd.to_datetime(time_df['ts']).dt.year
time_df['weekday'] = pd.to_datetime(time_df['ts']).dt.weekday

time_df.tail(5)

<ipython-input-29-0dd367ba8f77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df['timestamp'] = pd.to_datetime(time_df['ts']).dt.time
<ipython-input-29-0dd367ba8f77>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df['hour'] = pd.to_datetime(time_df['ts']).dt.hour
<ipython-input-29-0dd367ba8f77>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,ts,timestamp,hour,day,week,month,year,weekday
90,1970-01-01 00:25:41.970568796,00:25:41.970568,0,1,1,1,1970,3
91,1970-01-01 00:25:41.970828796,00:25:41.970828,0,1,1,1,1970,3
92,1970-01-01 00:25:41.971049796,00:25:41.971049,0,1,1,1,1970,3
93,1970-01-01 00:25:41.979540796,00:25:41.979540,0,1,1,1,1970,3
94,1970-01-01 00:25:41.979748796,00:25:41.979748,0,1,1,1,1970,3


In [30]:
time_df = time_df[['timestamp','hour','day','week','month','year','weekday']]
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,00:25:41.903636,0,1,1,1,1970,3
1,00:25:41.903770,0,1,1,1,1970,3
2,00:25:41.904034,0,1,1,1,1970,3
4,00:25:41.910973,0,1,1,1,1970,3
5,00:25:41.911006,0,1,1,1,1970,3


In [31]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  73 non-null     object
 1   hour       73 non-null     int64 
 2   day        73 non-null     int64 
 3   week       73 non-null     int64 
 4   month      73 non-null     int64 
 5   year       73 non-null     int64 
 6   weekday    73 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 4.6+ KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [32]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
user_df = df3[['userId','firstName','lastName','gender','level']]
user_df.head(1)

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [34]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.